The dataset is on kaggle and to export the data to our platform we use this code  "Given to us in Kagge"

About Dataset
The Hawaii Natural Energy Institute examined 14 NMC-LCO 18650 batteries with a nominal capacity of 2.8 Ah, which were cycled over 1000 times at 25°C with a CC-CV charge rate of C/2 rate and discharge rate of 1.5C.

From that source dataset, I created features that showcase the voltage and current behaviour over each cycle. Those features can be used to predict the remaining useful life (RUL) of the batteries. The dataset contains the summary of the 14 batteries.

Variables:

Cycle Index: number of cycle
F1: Discharge Time (s)
F2: Time at 4.15V (s)
F3: Time Constant Current (s)
F4: Decrement 3.6-3.4V (s)
F5: Max. Voltage Discharge (V)
F6: Min. Voltage Charge (V)
F7: Charging Time (s)
Total time (s)
RUL: target

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ignaciovinuales/battery-remaining-useful-life-rul")

print("Path to dataset files:", path)

The file is imported as a folder and we need to find the specific csv file within the data

In [ ]:
import os

#list the files in the downloaded directory and state how many csv files there are

downloaded_directory = path
files_in_directory = os.listdir(downloaded_directory)

print("Files in the downloaded directory:")
i = 0
for file_name in files_in_directory:
    if file_name.endswith(".csv"):
        i += 1
        print(file_name)

print("Number of CSV files:", i)



In [ ]:
import pandas as pd

HNEI_df = pd.read_csv(path+ '/Battery_RUL.csv')
HNEI_df.head()

In [ ]:
HNEI_df.info()

In [ ]:
HNEI_df.describe()

In [ ]:
HNEI_df.isnull().sum() #figure out if we have any missing data

In [ ]:
HNEI_df.duplicated().sum() #figure out if we have any duplicates

In [ ]:
HNEI_df.corr()

In [ ]:
import numpy as np
HNEI_df.corr()['RUL'].sort_values( key = np.abs, ascending= False) #it seems it's all in the rate of charge and discharge

In [ ]:
HNEI_df.value_counts('Cycle_Index')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

HNEI_df.hist(bins=50, figsize=(20,15))
plt.show()


In [ ]:
sns.pairplot(HNEI_df)
plt.show()

In [ ]:
sns.heatmap(HNEI_df.corr(), annot=True)


In [ ]:
HNEI_df.columns

In [ ]:
HNEI_df['Cycle_Index'].hist( bins=50, figsize=(20,15))
plt.show()

In [ ]:
Batter_indices = HNEI_df[HNEI_df['Cycle_Index']==1].index #There is 14 0f them I can group these batteries

In [ ]:
Batter_indices

In [ ]:
HNEI_df.loc[0:1076-1]

In [ ]:
x = 0
prev = 0
for value in Batter_indices:
  value = value -1
  Battery_df = HNEI_df.loc[prev :value]
  prev = value
  Battery_df.to_csv(f'Battery_{x}.csv', index=False)
  x += 1



In [ ]:
battery_1 = pd.read_csv('Battery_1.csv')
battery_1

In [ ]:
battery_id = 0

def add_id(Cycle_Index):
  global battery_id
  if Cycle_Index == 1:
    battery_id = battery_id +1
  return battery_id


HNEI_df['battery_id'] = HNEI_df['Cycle_Index'].apply(add_id)


In [ ]:
HNEI_df

In [ ]:
HNEI_df[HNEI_df['Cycle_Index']==1] #There is 14 0f them I can group these batteries

In [ ]:
HNEI_df.corr()

In [ ]:
for i in range (1,15):
  max = HNEI_df[HNEI_df['battery_id']==i]['RUL'].max()
  print(f'for battery {i} the max is {max}')

In [ ]:
#HNEI_df.drop(['Cycle_Index'], axis = 1, inplace = True)
HNEI_df

In [ ]:
# I want to split the data by removing a certain percentage of each battery at the end not randomly
x = HNEI_df.drop(['RUL'], axis = 1)
y = HNEI_df['RUL']

In [ ]:
x[x['battery_id'] == 1].tail(10).index

In [ ]:
#let's make the indices for the test sets

test_indices = []

for i in range (1,15):
   test_index = test_indices.append(x[x['battery_id'] == i].tail(200).index)



In [ ]:
# Flatten the list of Index objects into a single list of indices
flat_test_indices = [idx for index_list in test_indices for idx in index_list]

# Select the rows using the flattened list of indices
x.loc[flat_test_indices]

In [ ]:
y

In [ ]:
#Okay let's start of with a linear Regression Model

In [ ]:
# Select the corresponding rows for y using the flattened list of indices
y_test = y.loc[flat_test_indices]

# Select the test sets for x
x_test = x.loc[flat_test_indices]

# Create the training indices by dropping the test indices from the original index
train_indices = x.index.drop(flat_test_indices)

# Select the training sets for x and y
x_train = x.loc[train_indices]
y_train = y.loc[train_indices]

print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
x_test

In [ ]:
y_test

In [ ]:
#the model as you will see below is very terrible, but that is the first iteration I will pycaret to make the iterating faster from now on
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# Calculate the R-squared (R2) score
r2 = r2_score(y_test, predictions)

In [ ]:
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2) Score:", r2)

In [ ]:
HNEI_df.drop(['Cycle_Index'], axis = 1, inplace = True) # this will cause extreme data leakage but I also have the batteryID

In [ ]:
from pycaret.regression import *
exp_reg = setup(HNEI_df, target = 'RUL', verbose=False, session_id=1)

In [ ]:
%pip install pycaret

In [ ]:
# Compare different regression models
best_model = compare_models()

In [ ]:
# Create Extra Trees Regressor model
et = create_model('et')

In [ ]:
# Make predictions on the test set
predictions = predict_model(et)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a scatter plot of actual vs. predicted RUL
plt.figure(figsize=(10, 6))
sns.scatterplot(x='RUL', y='prediction_label', data=predictions)
plt.xlabel("Actual RUL")
plt.ylabel("Predicted RUL")
plt.title("Actual vs. Predicted RUL (Extra Trees Regressor)")
plt.show()

In [ ]:
rf = create_model('rf')
light = create_model('lightgbm')

In [ ]:
ensem = blend_models(estimator_list = [rf, light, et])